# API Deployment with FastAPI

This notebook demonstrates how to build and test a FastAPI endpoint for serving the fashion product classification model.

In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import torch
from torchvision import transforms
from PIL import Image
import pickle

# Load the trained model
model = torch.load('fashion_model.pth')
model.eval()

# Load label encoders
with open('encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Initialize FastAPI app
app = FastAPI()

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Load and preprocess the image
    image = Image.open(file.file)
    image = transform(image).unsqueeze(0)
    
    # Make prediction
    with torch.no_grad():
        output = model(image)
    
    # Decode predictions
    _, predicted = torch.max(output, 1)
    predicted_label = encoders['label_encoder'].inverse_transform(predicted.numpy())
    
    return JSONResponse(content={'predicted_label': predicted_label.tolist()})

# To run the app, use the command:
# !uvicorn filename:app --reload
# Replace 'filename' with the name of this notebook without the .ipynb extension.